In [1]:
import os
import pickle

import opencc
t2s = opencc.OpenCC('tw2sp.json')
t2s.convert('隨身碟')

'U盘'

In [2]:
from FlagEmbedding import BGEM3FlagModel

model = BGEM3FlagModel('BAAI/bge-m3',  use_fp16=True) 

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

...

onnx/config.json:   0%|          | 0.00/698 [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

sparse_linear.pt:   0%|          | 0.00/3.52k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

onnx/Constant_7_attr__value:   0%|          | 0.00/65.6k [00:00<?, ?B/s]

In [3]:
import pandas as pd
import json

ans_path="競賽資料集/dataset/preliminary/ground_truths_example.json"
with open(ans_path, 'rb') as f:
    ans = json.load(f)  # 讀取問題檔案

with open("競賽資料集/dataset/preliminary/questions_example.json", 'rb') as f:
    que = json.load(f) 
    
questions = pd.DataFrame( que['questions'] )

y=pd.DataFrame(ans['ground_truths'])
output=pd.merge(y,questions,on=['qid','category'])
output.loc[output[output['qid']==99].index[0],'retrieve']=693
output.loc[output[output['qid']==97].index[0],'retrieve']=579
output.loc[output[output['qid']==50].index[0],'source'].append(78)
output.loc[output[output['qid']==109].index[0],'source'].append(283)
output.loc[output[output['qid']==135].index[0],'source'].append(28)

# 訓練資料

In [6]:
faq=output[output['category'] == 'faq']

insurance=output[output['category'] == 'insurance']

finance=output[output['category'] == 'finance']

# 預測資料

In [5]:
faq=questions[questions['category'] == 'faq']

insurance=questions[questions['category'] == 'insurance']

finance=questions[questions['category'] == 'finance']

In [7]:
fag_key={}
with open(os.path.join('競賽資料集/reference', 'faq/pid_map_content.json'), 'rb') as f_s:
    key_to_source_dict = json.load(f_s)  # 讀取參考資料文件
    key_to_source_dict = {int(key): value for key, value in key_to_source_dict.items()}
    for key, value in key_to_source_dict.items():
        fag_key[int(key)]=[t2s.convert(v['question']) for v in value]

In [8]:
%%time

ans_list=[]
for row in faq.itertuples():
    sentences_1=[ t2s.convert(row.query) ]

    c_max=[]
    c_mean=[]
    for y_id in row.source:    
        sentences_2=fag_key[y_id]
        sentence_pairs = [[i,j] for i in sentences_1 for j in sentences_2]
        score_list=model.compute_score(sentence_pairs, 
                          max_passage_length=250,
                          weights_for_different_modes=[0.35, 0.3, 0.35])
        c_max.append(max(score_list['colbert+sparse+dense']))
        c_mean.append(sum(score_list['colbert+sparse+dense'])/len(score_list['colbert+sparse+dense']))
    
    ans_list.append({'qid':row.qid, 'category':row.category, 'retrieve':row.retrieve,'source':row.source,'c_max':c_max, 'c_mean':c_mean})
    
    # ans_list.append({'qid':row.qid, 'category':row.category ,'source':row.source,'c_max':c_max, 'c_mean':c_mean})
        # break
    # break

CPU times: user 24.2 s, sys: 3.02 s, total: 27.3 s
Wall time: 25.2 s


In [9]:
ans_df=pd.DataFrame(ans_list)

In [10]:


ans_df['c_m']= ans_df[['c_max','c_mean']].apply( lambda x: [a*0.7 + b*0.3 for a, b in zip(x['c_max'], x['c_mean'])] ,axis=1 )
ans_df['ans']=ans_df.apply( lambda x:  x['source'][x['c_m'].index(max(x['c_m']))] ,axis=1)

In [11]:
ans_df['score']=ans_df.apply(lambda x:1 if x['retrieve']==x['ans'] else 0,axis=1)
ans_df['score'].sum()

49

In [12]:
with open("llm_s_ans_insurance.pkl", "rb") as f:
    llm_s_ans=pickle.load(f)
with open("llm_ans_insurance.pkl", "rb") as f:
    llm_ans=pickle.load(f)

In [13]:
def parse_list(text):
    match = re.search(r'\[.*?\]', text, re.DOTALL)
    json_str = match.group(0)
    return eval(json_str)

In [14]:
%%time
import re
insurance_key={}
insurance_s_key={}

for key in llm_ans.keys():
    tmp1=[]
    tmp2=[]
    for v1, v2 in zip(llm_ans[key], llm_s_ans[key]):
        v1=t2s.convert(v1[1])
        v2=t2s.convert(v2[1])
        try:
            tmp1.extend(parse_list(v1))
        except:
            v1=v1.replace('未到期保险费会如何处理？','未到期保险费会如何处理？"]').replace('？」]','？"]')
            tmp1.extend(parse_list(v1))
            
        tmp2.extend(parse_list(v2))

    new_key=int(key.split('/')[-1].strip('.pdf'))
    insurance_key[new_key]=tmp1
    insurance_s_key[new_key]=tmp2

CPU times: user 825 ms, sys: 0 ns, total: 825 ms
Wall time: 825 ms


In [15]:
%%time
ans_list2=[]
for row in insurance.itertuples():
    sentences_1=[ t2s.convert(row.query) ]

    c_max=[]
    c_mean=[]
    for y_id in row.source:    
        sentences_2=insurance_s_key[y_id]+insurance_key[y_id]
        sentence_pairs = [[i,j] for i in sentences_1 for j in sentences_2]
        
        score_list=model.compute_score(sentence_pairs, 
                          max_passage_length=1024,
                          # weights_for_different_modes=[0.4, 0.3, 0.3],
                          weights_for_different_modes=[0, 0, 1])
        c_max.append(max(score_list['colbert+sparse+dense']))
        c_mean.append(sum(score_list['colbert+sparse+dense'])/len(score_list['colbert+sparse+dense']))

        # break
    # break    
    ans_list2.append({'qid':row.qid, 'category':row.category, 'retrieve':row.retrieve,'source':row.source,'c_max':c_max, 'c_mean':c_mean})

    # ans_list2.append({'qid':row.qid, 'category':row.category, 'source':row.source,'c_max':c_max, 'c_mean':c_mean})

Compute Scores: 100%|█████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  8.83it/s]


CPU times: user 26.4 s, sys: 4.12 s, total: 30.5 s
Wall time: 25.6 s


In [16]:
ans_df=pd.DataFrame(ans_list2)

In [17]:
ans_df['c_m']= ans_df[['c_max','c_mean']].apply( lambda x: [a*0.7 + b*0.3 for a, b in zip(x['c_max'], x['c_mean'])] ,axis=1 )
ans_df['ans']=ans_df.apply( lambda x:  x['source'][x['c_m'].index(max(x['c_m']))] ,axis=1)

In [18]:
ans_df['score']=ans_df.apply(lambda x:1 if x['retrieve']==x['ans'] else 0,axis=1)
ans_df['score'].sum()

47

In [27]:
ans_df['c_max'] = ans_df['c_max'].apply(lambda lst: [round(num, 2) for num in lst])
ans_df['c_mean'] = ans_df['c_mean'].apply(lambda lst: [round(num, 2) for num in lst])

In [28]:
ans_df[1!=ans_df['score']]

,qid,category,retrieve,source,c_max,c_mean,c_m,ans,score
3,4,insurance,186,"[186, 627, 536, 179, 174, 178]","[0.71, 0.75, 0.73, 0.71, 0.71, 0.71]","[0.53, 0.49, 0.53, 0.55, 0.52, 0.51]","[0.6584585005683558, 0.6715780609930064, 0.673...",536,0
28,29,insurance,578,"[527, 256, 102, 578, 524, 236, 431]","[0.8, 0.79, 0.83, 0.74, 0.78, 0.71, 0.79]","[0.67, 0.66, 0.64, 0.63, 0.67, 0.64, 0.66]","[0.7632008240140717, 0.753765582293272, 0.7707...",102,0
34,35,insurance,319,"[502, 62, 319, 224, 207, 283, 441]","[0.82, 0.71, 0.84, 0.91, 0.71, 0.83, 0.68]","[0.62, 0.61, 0.63, 0.68, 0.59, 0.63, 0.6]","[0.7630626180803919, 0.6765817351341247, 0.777...",224,0


In [29]:
# !pip install langchain

In [19]:
with open("finance_new_q.pkl", "rb") as f:
    finance_new_q=pickle.load(f)
finance_new_q=finance_new_q.set_index('qid')['new_query'].to_dict()

In [20]:
with open("corpus_dict_finance.pkl", "rb") as f:
    corpus_dict_finance=pickle.load(f)

with open("corpus_dict_finance_img.pkl", "rb") as f:
    corpus_dict_finance_img=pickle.load(f)

corpus_dict_finance.update(corpus_dict_finance_img)

In [23]:
# !pip install langchain

In [24]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=30)

In [25]:
%%time
ans_list3=[]
for row in finance.itertuples():
    
    sentences_1=[ t2s.convert(finance_new_q[row.qid]) ]
    
    c_max=[]
    for y_id in row.source:
        texts = text_splitter.create_documents([t2s.convert(corpus_dict_finance[y_id][:8000] )])
        sentences_2= [text.page_content for text in texts]

        sentence_pairs = [[i,j] for i in sentences_1 for j in sentences_2]
        score_list=model.compute_score(sentence_pairs, 
                          max_passage_length=600,
                          weights_for_different_modes=[0.2, 0.5, 0.3])
        c_max.append(max(score_list['colbert+sparse+dense']))

    c_mean=[]
    for y_id in row.source:
        sentences_2=[t2s.convert(corpus_dict_finance[y_id][:8000])]
        sentence_pairs = [[i,j] for i in sentences_1 for j in sentences_2]
        
        score_list=model.compute_score(sentence_pairs, 
                          max_passage_length=8000,
                          weights_for_different_modes=[0.2, 0.3, 0.5])
        c_mean.append(score_list['colbert+sparse+dense'][0])

    ans_list3.append({'qid':row.qid, 'category':row.category, 'retrieve':row.retrieve,'source':row.source,'c_max':c_max, 'c_mean':c_mean})
    
    # ans_list3.append({'qid':row.qid, 'category':row.category, 'source':row.source,'c_max':c_max, 'c_mean':c_mean})

    # break

CPU times: user 1min 24s, sys: 2.64 s, total: 1min 27s
Wall time: 1min 26s


In [26]:
ans_df=pd.DataFrame(ans_list3)

In [27]:
ans_df['c_m']= ans_df[['c_max','c_mean']].apply( lambda x: [a*0.4 + b*0.6 for a, b in zip(x['c_max'], x['c_mean'])] ,axis=1 )
ans_df['ans']=ans_df.apply( lambda x:  x['source'][x['c_m'].index(max(x['c_m']))] ,axis=1)

In [28]:
ans_df['score']=ans_df.apply(lambda x:1 if x['retrieve']==x['ans'] else 0,axis=1)
ans_df['score'].sum()

48

In [29]:
ans1=pd.DataFrame(ans_list2)
ans2=pd.DataFrame(ans_list3)
ans3=pd.DataFrame(ans_list)

In [30]:
all_ans=pd.concat([ans1,ans2,ans3])

In [31]:
import numpy as np

def one_model(category,x1,x2):
    x1=np.array(x1)
    x2=np.array(x2)
    if category=='insurance':
        return x1*0.7+x2*0.3
    if category=='finance':
        return x1*0.4+x2*0.6
    if category=='faq':
        return x1*0.7+x2*0.3

In [32]:
all_ans['c_m']=all_ans.apply(lambda x: one_model(x['category'],x['c_max'], x['c_mean']).tolist(),axis=1)
all_ans['ans']=all_ans.apply( lambda x:  x['source'][x['c_m'].index(max(x['c_m']))] ,axis=1)

In [33]:
all_ans['score']=all_ans.apply(lambda x:1 if x['retrieve']==x['ans'] else 0,axis=1)
all_ans['score'].sum()

144

In [34]:
all_ans['retrieve']=all_ans.apply( lambda x:  x['source'][x['c_m'].index(max(x['c_m']))] ,axis=1)

In [35]:
ans_output={'answers':all_ans[['qid','retrieve']].to_dict('records')}

In [36]:
with open('pred_retrieve.json', 'w', encoding='utf8') as f:
    json.dump(ans_output, f, ensure_ascii=False, indent=4)  # 儲存檔案，確保格式和非ASCII字符